In [1]:
from __future__ import print_function
from fenics import *
import numpy as np
import scipy
import random
import matplotlib.pyplot as plt
import copy
%matplotlib inline

In [9]:


"""
Example
2D random heat equation with forcing term f = 0
 
 
  du/dt = - div(a(ξ)∇u)   in the unit square
  u = 0                   on the boundary
  u = u_0                 at t = 0
 
  D = [0,1]^2  is physical space
  Τ = [-1,1]^M  is sample space

a(x,ξ) = 0.3 + sum_{m=1}^M (cos(2πmx1)+cos(2πmx2)/m^2π^2 * ξm)
u_0 = 10 sin(πx1) sin(πx2) + 4/3 sin(6πx1) sin(6πx2) + 2 sin(2πx1) sin(2πx2)ξ1 + 2 sin(4πx1) sin(4πx2)ξ2 + 2 sin(6πx1) sin(6πx2)(ξ1^2 − E[ξ1^2])

"""

class DLR:
    def __init__(self,dt,n,M,R = 3,sample_size = 50,mesh_type='2D',mean =None,U=None,Y=None,a=None,a_0=None,a_sto=None):
        self.dt = dt  # time step
        self.n = n  # number of mesh
        self.h = 1 / n  # element size
        self.M = M  # The number of random variables( The number of truncation of a(ξ))
        self.R = R  # rank of DLR
        self.sample_size = sample_size  # stochastic discretization
        
        if mesh_type == '1D':
            self.mesh = UnitIntervalMesh(self.n)
        elif mesh_type == '2D':
            self.mesh = UnitSquareMesh(self.n, self.n)
        else:
            raise ValueError("Invalid mesh_type. Use '1D' or '2D'.")

        self.V = FunctionSpace(self.mesh, 'P', 1)
        # self.V_R = MixedFunctionSpace([self.V] * self.R)

        ## Define boundary condition
        self.bc = DirichletBC(self.V, Constant(0), 'on_boundary')

        
        ## Initialize functions 
        #set random values
        self.sampled = self.sampling() #(M,self.sample_size) ,unifrom random values in T
        ksi_1 = self.sampled[0]
        ksi_2 = self.sampled[1]

        # E[u]
        if mean == None:
            mean = Expression('10 * sin(pi * x[0]) * sin(pi * x[1]) + 4/3 * sin(6 * pi * x[0]) * sin(6 * pi * x[1])',degree=3)
        self.mean = interpolate(mean,self.V)  #(V_h)
        self.mean_n = interpolate(mean,self.V) 
        
         # stochatic basis functions   
        if Y == None:
            self.Y = [] #(R,sample_size) # R=3
            Y_1 = ksi_1
            self.Y.append(Y_1)
            Y_2 = ksi_2
            self.Y.append(Y_2)
            Y_3 = ksi_1 ** 2 - 2/3
            self.Y.append(Y_3)
        else:
            self.Y = Y #(R,sample_size)
        self.Y = np.array(self.Y)
        
        # deterministic basis functions
        if U == None:
            U = [Expression('2 * sin(2 * pi * x[0]) * sin(2 * pi * x[1])',degree=3),Expression('2 * sin(4 * pi * x[0]) * sin(4 * pi * x[1])',degree=3),Expression('2 * sin(6 * pi * x[0]) * sin(6 * pi * x[1])',degree=3)]
        self.U = []#  (R,V_h) 
        self.U_n = []
        for u_i in U:
            self.U.append(interpolate(u_i,self.V))
            self.U_n.append(interpolate(u_i,self.V))
        


        ## set coefficient
        if a == None:
            self.a_0 = Constant(0.3) #(L^♾️)
            self.a_sto = [] #(sample_size,L^♾️)
            self.a = [] #(sample_size,L^♾️)
            for i in range(self.sample_size):
                str_expr = ""
                for j in range(1, self.M + 1):
                    str_expr += f"( cos(2 * pi * {j} * x[0] ) + cos(2 * pi * {j} * x[1] ) ) / ({j} * {j} * pi * pi) * sampled[{j-1}] + "
                str_expr = str_expr[:-3]
                self.a_sto.append(Expression(str_expr, degree=3, sampled=Constant(self.sampled[:,i])))
                str_expr += " + a_0"
                self.a.append(Expression(str_expr, degree=3, sampled=Constant(self.sampled[:,i]),a_0 = Constant(0.3)))
            # for i in range(self.sample_size):
            #     self.a[i] = interpolate(self.a[i],self.V)
        else:
            self.a_0 = a_0
            self.a_sto = a_sto
            self.a = a
        
#         print("hi",self.energynorm())
#         print("hi",self.exl2norm())
#         print(np.mean(self.Y[0]))
#         print(np.mean(self.Y[1]))
#         print(np.mean(self.Y[2]))

        for i in range(self.R):
            Y_i_mean = np.mean(self.Y[i])
            self.mean.vector()[:] += self.U[i].vector()[:] * Y_i_mean
            self.Y[i] -= Y_i_mean   
        self.mean_n.assign(self.mean) 
        
        self.reorthogonalize()
        
#         print("hi",self.energynorm())
#         print("hi",self.exl2norm())
#         print(np.matmul(self.Y,np.transpose(self.Y)))
#         print(np.mean(self.Y[0]))
#         print(np.mean(self.Y[1]))
#         print(np.mean(self.Y[2]))
        
        self.matrix = np.zeros((R,R))
                   
        ##for plot
        self.timelist = []
        self.energylist = []
        self.L2list= []

    
    def sampling(self):
        return np.random.uniform(low=-1.0, high=1.0, size=(self.M,self.sample_size))

    # calculate quadratures, M_i,j = <U_i,U_j>
    def matrix_calculate(self):
        for i in range(self.R):
            for j in range(i,self.R):
                value = assemble((self.U[i]*self.U[j]) * dx)
                self.matrix[i][j] = value
                self.matrix[j][i] = value

    ## subfunctions for calculating dinamics

    def E_a_grad_u(self,a):
        grad_list = []
        for i in range(self.sample_size):
            func = self.mean_n
            for j in range(self.R):
                func += (self.U_n[j] * self.Y[j][i])
            grad_list.append(grad(func))
        ans = a[0]* grad_list[0]
        for i in range(1,self.sample_size):
            ans += (a[i]* grad_list[i])
        ans /= self.sample_size
        return ans
    

    def E_a_grad_u_Y(self,a,Y):
        grad_list = []
        for i in range(self.sample_size):
            func = self.mean_n
            for j in range(self.R):
                func += (self.U_n[j] * self.Y[j][i])
            grad_list.append(grad(func))
        ans = a[0]* grad_list[0]* Y[0]
        for i in range(1,self.sample_size):
            ans += (a[i]* grad_list[i]* Y[i])
        ans /= self.sample_size
        return ans
    
    

    

    def a_grad_u_grad_U(self,a):
        ans = np.zeros((self.R,self.sample_size)) 
        u = []
        for i in range(self.sample_size):
            func = self.mean
            for j in range(self.R):
                func += (self.U[j] * Constant(self.Y[j][i]) )
            u.append(func)
        for i in range(self.R):
            for j in range(self.sample_size):
                ans[i][j] = assemble(a[j] * dot(grad(u[j]),grad(self.U[i])) * dx)
        row_means = np.mean(ans, axis=1)
        ans -= row_means[:, np.newaxis]

        return ans
    
    def dt_a0_grad_U_grad_U(self):
        ans = np.zeros((self.R,self.R)) 
        for i in range(self.R):
            for j in range(i,self.R):
                value = self.dt *  assemble(self.a_0 * dot(grad(self.U[i]),grad(self.U[j])) * dx)
                ans[i][j] = value
                ans[j][i] = value
        return ans
    


    # def pre_assemble(self):
    #     self.u_1 = Function(self.V)
    #     self.u_2 = Function(self.V)
    #     self.u_3 = Function(self.V)
    #     self.agradugradv = self.u_3 * dot(grad(self.u_1),grad(self.u_2)) * dx
    #     assemble(self.a)

    # def a_grad_u_grad_U(self,a):
    #     ans = np.zeros((self.R,self.sample_size)) 
    #     u = []
    #     for i in range(self.sample_size):
    #         u.append(self.mean + self.U[0] * Constant(self.Y[0][i]) + self.U[1] * Constant(self.Y[1][i]) + self.U[2] * Constant(self.Y[2][i]))
    #     for i in range(self.R):
    #         for j in range(self.sample_size):
    #             ans[i][j] = assemble(self.agradugradv(coefficients={self.u_3=a[j],self.u_1 = u[j],self.u_2 = self.U[i]}))
    #                 # self.a[j] * dot(grad(u[j]),grad(self.U[i])) * dx)
    #     return ans
                             
                          

    def orthogonal_projection(self,v):
        ortho = np.inner(v,self.Y[0]) / self.sample_size * self.Y[0]
        for i in range(1,self.R):
            ortho += np.inner(v,self.Y[i]) / self.sample_size * self.Y[i]
        return v - ortho
    
    def reorthogonalize(self): #dimension :Y(self.R,self.sample_size), U(self.R,V)
        Q, _ = np.linalg.qr(np.transpose(self.Y))
        # self.Y = np.transpose(Q) 
        self.Y = np.transpose(Q) *np.sqrt(self.sample_size)
        vectors = []
        for i in range(self.R):
            vectors.append(self.U[i].vector()[:])
        vectors = np.matmul(_,vectors)
        for i in range(self.R):
            # self.U[i].vector()[:] = vectors[i] 
            self.U[i].vector()[:] = vectors[i] /np.sqrt(self.sample_size)




    #explicit scheme
    def explicit_simulate(self,end = 2.5):
        t = 0
        count = 0 # calculate energynorm each step is cotly so only every some steps
        self.timelist.append(t)
        energy = self.energynorm()
        self.energylist.append(energy)
        l2 = self.exl2norm()
        self.L2list.append(l2)
        print("time: ",t)
        print("energy norm: ", energy )
        print("L2 norm: ", l2 )

        # Define variational problem
        
        self.mean = TrialFunction(self.V)
        for i in range(self.R):
            self.U[i] = TrialFunction(self.V)
        v = TestFunction(self.V)

        a_mean = self.mean * v * dx  
        L_mean = self.mean_n * v * dx - self.dt * dot(self.E_a_grad_u(self.a), grad(v)) * dx
        lhs = []
        rhs = []
        for i in range(self.R):
            a_i = self.U[i] * v * dx
            L_i = self.U_n[i] * v * dx - self.dt * dot(self.E_a_grad_u_Y(self.a,self.Y[i]), grad(v)) * dx
            lhs.append(a_i)
            rhs.append(L_i)
           
        self.mean = Function(self.V)
        for i in range(self.R):
            self.U[i] = Function(self.V)
        
        while t < end:           
            # Compute solution
            solve(a_mean == L_mean, self.mean, self.bc) 
            for i in range(self.R):
                solve(lhs[i]==rhs[i],self.U[i],self.bc)
         
            self.matrix_calculate()

            det = np.linalg.det(self.matrix)
#             if np.isclose(det, 0):
#                 break
#             else:
            A = []
            for i in range(self.R):
                A.append(self.orthogonal_projection(self.a_grad_u_grad_U(self.a)[i]))
            A = np.array(A)
            self.Y += -self.dt * scipy.linalg.solve(self.matrix,A)
                #                 self.Y += -self.dt * np.matmul(scipy.linalg.inv(self.matrix),A)
                
            
            # reorthogonalize
            self.reorthogonalize()

            t  += self.dt
            count += 1
            
            self.mean_n.assign(self.mean) 
            for i in range(self.R):
                self.U_n[i].assign(self.U[i])
           
            if count % 1 == 0:
                self.timelist.append(t)
                energy = self.energynorm()
                self.energylist.append(energy)
                l2 = self.exl2norm()
                self.L2list.append(l2)
                print("time: ",t)
                print("energy norm: ", energy )
                print("L2 norm: ", l2 )
                if energy > 10 ** 7:
                    break 
                    
        self.plot_field()
        self.plot_norm()

        

    #semi implicit scheme
    def semi_implicit_simulate(self,end = 2.5):
        t = 0
        count = 0 # calculate energynorm each step is cotly so only every some steps
        self.timelist.append(t)
        energy = self.energynorm()
        self.energylist.append(energy)
        l2 = self.exl2norm()
        self.L2list.append(l2)
        print("time: ",t)
        print("energy norm: ", energy )
        print("L2 norm: ", l2 )

        # Define variational problem
        
        self.mean = TrialFunction(self.V)
        for i in range(self.R):
            self.U[i] = TrialFunction(self.V)
        v = TestFunction(self.V)

        a_mean = self.mean * v * dx  + self.dt * self.a_0 * dot(grad(self.mean),grad(v)) * dx
        L_mean = self.mean_n * v * dx - self.dt * dot(self.E_a_grad_u(self.a_sto),grad(v)) * dx
        lhs = []
        rhs = []
        for i in range(self.R):
            a_i = self.U[i] * v * dx + self.dt * self.a_0 * dot(grad(self.U[i]),grad(v)) * dx
            L_i = self.U_n[i] * v * dx - self.dt * dot(self.E_a_grad_u_Y(self.a_sto,self.Y[i]), grad(v)) * dx
            lhs.append(a_i)
            rhs.append(L_i)
   
        self.mean = Function(self.V)
        for i in range(self.R):
            self.U[i] =  Function(self.V)

        while t < end:           
            # Compute solution
            solve(a_mean == L_mean, self.mean, self.bc) 
            for i in range(self.R):
                solve(lhs[i]==rhs[i],self.U[i],self.bc)
         
            self.matrix_calculate()
            
            A = []
            for i in range(self.R):
                A.append(self.orthogonal_projection(self.a_grad_u_grad_U(self.a_sto)[i]))
            A = np.array(A)
            self.Y += -self.dt * scipy.linalg.solve(self.matrix+ self.dt_a0_grad_U_grad_U(),A)
            
            #             self.Y += -self.dt * np.matmul(scipy.linalg.inv(self.matrix + self.dt_a0_grad_U_grad_U()),A)
            
            # reorthogonalize
            self.reorthogonalize()

            t  += self.dt
            count += 1
            
            self.mean_n.assign(self.mean) 
            for i in range(self.R):
                self.U_n[i].assign(self.U[i])
            
            if count % 1 == 0:
                self.timelist.append(t)
                energy = self.energynorm()
                self.energylist.append(energy)
                l2 = self.exl2norm()
                self.L2list.append(l2)
                print("time: ",t)
                print("energy norm: ", energy )
                print("L2 norm: ", l2 )
                if energy > 10 ** 7:
                    break 
                    
        self.plot_field()
        self.plot_norm()

        
        

   #energy norm

    def energynorm(self):
        u = self.mean 
        for j in range(self.R):
            u += self.U[j] * Constant(self.Y[j][0])
        form = self.a[0] * inner(grad(u), grad(u)) * dx
        for i in range(1,self.sample_size):
            u = self.mean
            for j in range(self.R):
                u += self.U[j] * Constant(self.Y[j][i])
            form += self.a[i] * inner(grad(u), grad(u)) * dx
        energy = assemble(form) / self.sample_size
        return energy
    
    def exl2norm(self):
        u = self.mean
        for j in range(self.R):
            u += self.U[j] * Constant(self.Y[j][0])
        form = u* u * dx
        for i in range(1,self.sample_size):
            u = self.mean
            for j in range(self.R):
                u += self.U[j] * Constant(self.Y[j][i])
            form += u* u * dx
        exl2 = assemble(form) / self.sample_size
        return exl2

    
    # monitor energy norm
    def plot_norm(self):
            plt.plot(self.timelist, self.energylist)
            plt.plot(self.timelist, self.L2list)
            plt.yscale('log')
            plt.xlabel("time")
            plt.ylabel("norm")
            plt.legend(['energy norm','L2 norm'])
            plt.show()
        
    #visualize mean function of the random field
    def plot_field(self):
        # u = self.mean + self.U[0] * np.mean(self.Y[0]) + self.U[1] * np.mean(self.Y[1]) + self.U[2] * np.mean(self.Y[2])
        plot(self.mean)
        plt.show()


Not separate mean

In [3]:


"""
Example
2D random heat equation with forcing term f = 0
 
 
  du/dt = - div(a(ξ)∇u)   in the unit square
  u = 0                   on the boundary
  u = u_0                 at t = 0
 
  D = [0,1]^2  is physical space
  Τ = [-1,1]^M  is sample space

a(x,ξ) = 0.3 + sum_{m=1}^M (cos(2πmx1)+cos(2πmx2)/m^2π^2 * ξm)
u_0 = 10 sin(πx1) sin(πx2) + 4/3 sin(6πx1) sin(6πx2) + 2 sin(2πx1) sin(2πx2)ξ1 + 2 sin(4πx1) sin(4πx2)ξ2 + 2 sin(6πx1) sin(6πx2)(ξ1^2 − E[ξ1^2])

"""

class DLR2:
    def __init__(self,dt,n,M,U,Y,a,a_0,a_sto,R = 3,sample_size = 50,mesh_type='2D'):
        self.dt = dt  # time step
        self.n = n  # number of mesh
        self.h = 1 / n  # element size
        self.M = M  # The number of random variables( The number of truncation of a(ξ))
        self.R = R  # rank of DLR
        self.sample_size = sample_size  # stochastic discretization
        
        if mesh_type == '1D':
            self.mesh = UnitIntervalMesh(self.n)
        elif mesh_type == '2D':
            self.mesh = UnitSquareMesh(self.n, self.n)
        else:
            raise ValueError("Invalid mesh_type. Use '1D' or '2D'.")

        self.V = FunctionSpace(self.mesh, 'P', 1)
        # self.V_R = MixedFunctionSpace([self.V] * self.R)

        ## Define boundary condition
        self.bc = DirichletBC(self.V, Constant(0), 'on_boundary')

        
        ## Initialize functions 
       
         # stochatic basis functions        
        self.Y = Y #(R,sample_size)
        self.Y = np.array(self.Y)
        
        # deterministic basis functions
        self.U = []#  (R,V_h) 
        self.U_n = []
        for u_i in U:
            self.U.append(interpolate(u_i,self.V))
            self.U_n.append(interpolate(u_i,self.V))

        ## set coefficient
        self.a_0 = a_0
        self.a_sto = a_sto
        self.a = a 
        
#         print("hi",self.energynorm())
#         print("hi",self.exl2norm())
#         print(np.mean(self.Y[0]))
#         print(np.mean(self.Y[1]))
#         print(np.mean(self.Y[2]))
       
        self.reorthogonalize()
#         print("hi",self.energynorm())
#         print("hi",self.exl2norm())
#         print(np.matmul(self.Y,np.transpose(self.Y)))
#         print(np.mean(self.Y[0]))
#         print(np.mean(self.Y[1]))
#         print(np.mean(self.Y[2]))
       
        
        self.matrix = np.zeros((R,R))

        
                   
        ##for plot
        self.timelist = []
        self.energylist = []
        self.L2list= []

   
    # calculate quadratures, M_i,j = <U_i,U_j>
    def matrix_calculate(self):
        for i in range(self.R):
            for j in range(i,self.R):
                value = assemble((self.U[i]*self.U[j]) * dx)
                self.matrix[i][j] = value
                self.matrix[j][i] = value

    ## subfunctions for calculating dinamics
    

    def E_a_grad_u_Y(self,a,Y):
        grad_list = []
        for i in range(self.sample_size):
            func = Constant(0)
            for j in range(self.R):
                func += (self.U_n[j] * self.Y[j][i])
            grad_list.append(grad(func))
        ans = a[0]* grad_list[0]* Y[0]
        for i in range(1,self.sample_size):
            ans += (a[i]* grad_list[i]* Y[i])
        ans /= self.sample_size
        return ans
    
    

    

    def a_grad_u_grad_U(self,a):
        ans = np.zeros((self.R,self.sample_size)) 
        u = []
        for i in range(self.sample_size):
            func = Constant(0)
            for j in range(self.R):
                func += (self.U[j] * Constant(self.Y[j][i]) )
            u.append(func)
        for i in range(self.R):
            for j in range(self.sample_size):
                ans[i][j] = assemble(dot(a[j] *grad(u[j]),grad(self.U[i])) * dx)
        
        return ans
    
    def dt_a0_grad_U_grad_U(self):
        ans = np.zeros((self.R,self.R)) 
        for i in range(self.R):
            for j in range(i,self.R):
                value = self.dt *  assemble(self.a_0 * dot(grad(self.U[i]),grad(self.U[j])) * dx)
                ans[i][j] = value
                ans[j][i] = value
        return ans
    


    # def pre_assemble(self):
    #     self.u_1 = Function(self.V)
    #     self.u_2 = Function(self.V)
    #     self.u_3 = Function(self.V)
    #     self.agradugradv = self.u_3 * dot(grad(self.u_1),grad(self.u_2)) * dx
    #     assemble(self.a)

    # def a_grad_u_grad_U(self,a):
    #     ans = np.zeros((self.R,self.sample_size)) 
    #     u = []
    #     for i in range(self.sample_size):
    #         u.append(self.mean + self.U[0] * Constant(self.Y[0][i]) + self.U[1] * Constant(self.Y[1][i]) + self.U[2] * Constant(self.Y[2][i]))
    #     for i in range(self.R):
    #         for j in range(self.sample_size):
    #             ans[i][j] = assemble(self.agradugradv(coefficients={self.u_3=a[j],self.u_1 = u[j],self.u_2 = self.U[i]}))
    #                 # self.a[j] * dot(grad(u[j]),grad(self.U[i])) * dx)
    #     return ans
                             
                          

    def orthogonal_projection(self,v):
        ortho = np.inner(v,self.Y[0]) / self.sample_size * self.Y[0]
        for i in range(1,self.R):
            ortho += np.inner(v,self.Y[i]) / self.sample_size * self.Y[i]
        return v - ortho
    
    def reorthogonalize(self):
        Q, _ = np.linalg.qr(np.transpose(self.Y))
        # self.Y =  np.transpose(Q)
        self.Y =  np.sqrt(self.sample_size) * np.transpose(Q)
        vectors = []
        for i in range(self.R):
            vectors.append(self.U[i].vector()[:])
        # vectors = np.matmul(_ ,vectors)
        vectors = np.matmul(_ /np.sqrt(self.sample_size),vectors)
        for i in range(self.R):
            self.U[i].vector()[:] = vectors[i]


    #explicit scheme
    def explicit_simulate(self,end = 2.5):
        t = 0
        count = 0 # calculate energynorm each step is cotly so only every some steps
        self.timelist.append(t)
        energy = self.energynorm()
        self.energylist.append(energy)
        l2 = self.exl2norm()
        self.L2list.append(l2)
        print("time: ",t)
        print("energy norm: ", energy )
        print("L2 norm: ", l2 )

        # Define variational problem
        
        for i in range(self.R):
            self.U[i] = TrialFunction(self.V)
        v = TestFunction(self.V)

        lhs = []
        rhs = []
        for i in range(self.R):
            a_i = self.U[i] * v * dx
            L_i = self.U_n[i] * v * dx - self.dt * dot(self.E_a_grad_u_Y(self.a,self.Y[i]), grad(v)) * dx
            lhs.append(a_i)
            rhs.append(L_i)
           
        for i in range(self.R):
            self.U[i] = Function(self.V)
        
        while t < end:           
            # Compute solution
            for i in range(self.R):
                solve(lhs[i]==rhs[i],self.U[i],self.bc)
         
            self.matrix_calculate()

            det = np.linalg.det(self.matrix)
            if np.isclose(det, 0):
                break
            else:
                A = []
                for i in range(self.R):
                    A.append(self.orthogonal_projection(self.a_grad_u_grad_U(self.a)[i]))
                A = np.array(A)
                self.Y += -self.dt * scipy.linalg.solve(self.matrix,A)
                #                 self.Y += -self.dt * np.matmul(scipy.linalg.inv(self.matrix),A)
                
            
            # reorthogonalize
            self.reorthogonalize()

            t  += self.dt
            count += 1
            
            for i in range(self.R):
                self.U_n[i].assign(self.U[i])
           
            if count % 1 == 0:
                self.timelist.append(t)
                energy = self.energynorm()
                self.energylist.append(energy)
                l2 = self.exl2norm()
                self.L2list.append(l2)
                print("time: ",t)
                print("energy norm: ", energy )
                print("L2 norm: ", l2 )
                if energy > 10 ** 7:
                    break 
                    
        self.plot_field()
        self.plot_norm()

        

    #semi implicit scheme
    def semi_implicit_simulate(self,end = 2.5):
        t = 0
        count = 0 # calculate energynorm each step is cotly so only every some steps
        self.timelist.append(t)
        energy = self.energynorm()
        self.energylist.append(energy)
        l2 = self.exl2norm()
        self.L2list.append(l2)
        print("time: ",t)
        print("energy norm: ", energy )
        print("L2 norm: ", l2 )

        # Define variational problem
        
        for i in range(self.R):
            self.U[i] = TrialFunction(self.V)
        v = TestFunction(self.V)

        
        lhs = []
        rhs = []
        for i in range(self.R):
            a_i = self.U[i] * v * dx + self.dt * self.a_0 * dot(grad(self.U[i]),grad(v)) * dx
            L_i = self.U_n[i] * v * dx - self.dt * dot(self.E_a_grad_u_Y(self.a_sto,self.Y[i]), grad(v)) * dx
            lhs.append(a_i)
            rhs.append(L_i)
   
        for i in range(self.R):
            self.U[i] =  Function(self.V)

        while t < end:           
            # Compute solution
            for i in range(self.R):
                solve(lhs[i]==rhs[i],self.U[i],self.bc)
         
            self.matrix_calculate()
            
            A = []
            for i in range(self.R):
                A.append(self.orthogonal_projection(self.a_grad_u_grad_U(self.a_sto)[i]))
            A = np.array(A)
            self.Y += -self.dt * scipy.linalg.solve(self.matrix+ self.dt_a0_grad_U_grad_U(),A)
            
            #             self.Y += -self.dt * np.matmul(scipy.linalg.inv(self.matrix + self.dt_a0_grad_U_grad_U()),A)
            
            # reorthogonalize
            self.reorthogonalize()

            t  += self.dt
            count += 1
             
            for i in range(self.R):
                self.U_n[i].assign(self.U[i])
            
            if count % 1 == 0:
                self.timelist.append(t)
                energy = self.energynorm()
                self.energylist.append(energy)
                l2 = self.exl2norm()
                self.L2list.append(l2)
                print("time: ",t)
                print("energy norm: ", energy )
                print("L2 norm: ", l2 )
                if energy > 10 ** 7:
                    break 
                    
        self.plot_field()
        self.plot_norm()

        
        

   #energy norm

    def energynorm(self):
        u = Constant(0)
        for j in range(self.R):
            u += self.U[j] * Constant(self.Y[j][0])
        form = self.a[0] * inner(grad(u), grad(u)) * dx
        for i in range(1,self.sample_size):
            u = Constant(0)
            for j in range(self.R):
                u += self.U[j] * Constant(self.Y[j][i])
            form += self.a[i] * inner(grad(u), grad(u)) * dx
        energy = assemble(form) / self.sample_size
        return energy
    
    def exl2norm(self):
        u = Constant(0)
        for j in range(self.R):
            u += self.U[j] * Constant(self.Y[j][0])
        form = u* u * dx
        for i in range(1,self.sample_size):
            u = Constant(0)
            for j in range(self.R):
                u += self.U[j] * Constant(self.Y[j][i])
            form += u* u * dx
        exl2 = assemble(form) / self.sample_size
        return exl2
            


    
    # monitor energy norm
    def plot_norm(self):
            plt.plot(self.timelist, self.energylist)
            plt.plot(self.timelist, self.L2list)
            plt.yscale('log')
            plt.xlabel("time")
            plt.ylabel("norm")
            plt.legend(['energy norm','L2 norm'])
            plt.show()
        
    #visualize mean function of the random field
    def plot_field(self):
        u = 0
        for i in range(self.R):
            u += self.U[i] * np.mean(self.Y[i])
        plot(u)
        plt.show()
        


    

   
    



In [ ]:
random_heat = DLR(dt = 0.0017/3,n = 7,M = 10)
random_heat.explicit_simulate(end=1)

time:  0
energy norm:  191.39975372065197
L2 norm:  23.870542528699424
Calling FFC just-in-time (JIT) compiler, this may take some time.
Solving linear variational problem.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Solving linear variational problem.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Solving linear variational problem.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Solving linear variational problem.
time:  0.0005666666666666666
energy norm:  297.7298152409025
L2 norm:  24.586953632181544
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.0011333333333333332
energy norm:  259.78281997186747
L2 norm:  24.260731610364818
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.0016999999999999997
energy norm:  233.04944

time:  0.01983333333333333
energy norm:  119.83314342274257
L2 norm:  18.562086892188372
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.020399999999999998
energy norm:  118.78270911100508
L2 norm:  18.426567355080664
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.020966666666666665
energy norm:  117.74932514815582
L2 norm:  18.292233777244668
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.02153333333333333
energy norm:  116.7321437607103
L2 norm:  18.159067063536188
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.022099999999999998
energy norm:  115.73041102564478
L2 nor

time:  0.04023333333333333
energy norm:  89.48379180779082
L2 norm:  14.327504227804985
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.040799999999999996
energy norm:  88.80214378495327
L2 norm:  14.226277430571304
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.04136666666666666
energy norm:  88.12717428007849
L2 norm:  14.125821238943752
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.04193333333333333
energy norm:  87.45877389080866
L2 norm:  14.026128117709108
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.042499999999999996
energy norm:  86.79683623391213
L2 norm:  1

time:  0.06063333333333333
energy norm:  68.51266994213934
L2 norm:  11.121207486089562
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.0612
energy norm:  68.01876043136411
L2 norm:  11.043694597374918
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.061766666666666664
energy norm:  67.52889644244019
L2 norm:  10.966740338788286
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.06233333333333333
energy norm:  67.043030734607
L2 norm:  10.8903401390927
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.0629
energy norm:  66.5611169886441
L2 norm:  10.814489480306895
Solving linear

time:  0.08103333333333311
energy norm:  52.98941271166785
L2 norm:  8.652837037409194
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.08159999999999977
energy norm:  52.616726576173825
L2 norm:  8.592883363322473
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.08216666666666643
energy norm:  52.24682392666321
L2 norm:  8.533351294317317
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.08273333333333309
energy norm:  51.87967919967878
L2 norm:  8.474237683041544
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.08329999999999975
energy norm:  51.515267199858684
L2 norm:  8.415

...

time:  0.8483000000000246
energy norm:  0.0045209421620730425
L2 norm:  0.0007435901386905558
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.8488666666666913
energy norm:  0.004489825172585424
L2 norm:  0.0007384735903157685
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.849433333333358
energy norm:  0.004458922412028093
L2 norm:  0.0007333922572863961
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.8500000000000247
energy norm:  0.004428232405130266
L2 norm:  0.0007283458971651377
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.8505666666666915
energy norm:  0.0043977536

time:  0.8681333333333596
energy norm:  0.003550168210670013
L2 norm:  0.0005839611297606328
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.8687000000000263
energy norm:  0.00352573444328706
L2 norm:  0.0005799432177631808
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.869266666666693
energy norm:  0.003501468882714023
L2 norm:  0.0005759529578583421
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.8698333333333598
energy norm:  0.0034773703706816868
L2 norm:  0.0005719901596895282
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.8704000000000265
energy norm:  0.00345343775

time:  0.8879666666666947
energy norm:  0.0027878890101038097
L2 norm:  0.0004586071195641192
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.8885333333333614
energy norm:  0.002768702767225122
L2 norm:  0.00045545189433695443
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.8891000000000281
energy norm:  0.0027496485978625884
L2 norm:  0.0004523183826549028
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.8896666666666948
energy norm:  0.0027307255926180595
L2 norm:  0.00044920643505171466
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.8902333333333615
energy norm:  0.00271

time:  0.9078000000000297
energy norm:  0.002189316650694536
L2 norm:  0.00036016720054844785
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.9083666666666964
energy norm:  0.002174250716551148
L2 norm:  0.0003576893968197956
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.9089333333333631
energy norm:  0.002159288486190844
L2 norm:  0.0003552286437094646
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.9095000000000298
energy norm:  0.002144429245601854
L2 norm:  0.00035278482385610365
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.9100666666666966
energy norm:  0.00212967

time:  0.9276333333333647
energy norm:  0.0017192863427329
L2 norm:  0.00028286161995470625
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.9282000000000314
energy norm:  0.0017074556875150218
L2 norm:  0.00028091576702936777
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.9287666666666982
energy norm:  0.0016957064614911658
L2 norm:  0.0002789833033517703
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.9293333333333649
energy norm:  0.0016840381040479921
L2 norm:  0.0002770641367680015
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
time:  0.9299000000000316
energy norm:  0.0016724